### **3. Wrangle, prepare, cleanse the data**

In this section, we will prepare the data to ensure that the model operates optimally. We will follow a set of steps based on the analysis we conducted in Section 2 Data Analysis. The phases we will work on include:

**3.1 Cleaning:** Removing duplicates, correcting incorrect column names, removing irrelevant columns and changing data types.\
**3.2 Integration:** Data grouping and handling missing values.\
**3.3 Construction:** Creating new variables and encoding categorical variables.\
**3.4 Variable selection:** Studying correlations and variance.

Libraries:

In [42]:
# Data analysis and wrangling
import numpy as np
import pandas as pd
import datetime
import time
from sklearn.preprocessing import LabelEncoder,MinMaxScaler,OrdinalEncoder,StandardScaler
from sklearn.feature_selection import VarianceThreshold

# Visualization
import seaborn as sns
import matplotlib.pyplot as plt

# Machine learning

# Best practices
pd.set_option("display.max_rows", None)
pd.set_option('display.max_columns',None)  
pd.options.display.float_format = '{:.2f}'.format


Acquire data:

In [3]:
df = pd.read_csv('data_easymoney.csv')

#### **3.1 Cleaning**

##### **Removing duplicates**

We note that we do not have any duplicated rows.

In [4]:
df.duplicated().sum()

0

##### **Correcting incorrect column names**

We can see how the em_acount variable is misspelled, it should be em_account. Not correcting this error, no matter how insignificant it may seem now, can lead to problems later when working with this column. We proceed to correct the error:

In [5]:
df.columns

Index(['Unnamed: 0', 'pk_cid', 'pk_partition', 'short_term_deposit', 'loans',
       'mortgage', 'funds', 'securities', 'long_term_deposit', 'em_account_pp',
       'credit_card', 'payroll', 'pension_plan', 'payroll_account',
       'emc_account', 'debit_card', 'em_account_p', 'em_acount', 'entry_date',
       'entry_channel', 'active_customer', 'segment', 'country_id',
       'region_code', 'gender', 'age', 'deceased', 'salary'],
      dtype='object')

In [6]:
df.rename(columns = {"em_acount":"em_account"}, inplace = True)

##### **Removing inrrelevant columns**

Although later we will do correlation and variance studies, where we will delve into those columns that may be irrelevant, taking a quick look we can see that:

* 'Unnamed:0' : This column acts as an inndex of the rows of our sample. This column is obviusly irrelevant since the dataframe structure already has indexes.
* 'em_account_pp': All the values in the 'em_account_pp' column are 0`s. Obviously this column is irrelevant because it is not giving us any new information.

In [7]:
df['em_account_pp'].value_counts()

em_account_pp
0    5962924
Name: count, dtype: int64

In [8]:
df.drop(columns = ["Unnamed: 0", "em_account_pp"], axis = 1, inplace = True)

##### **Changing data types**

We have two columns that inform us of dates, so the optimal thing is to convert them to datetime. But first of all, we have to change those dates that can be problematic, such as leap years. In our data we have two leap years:

In [9]:
df.loc[df['entry_date'] == '2015-02-29', 'entry_date'] = '2015-02-28'
df.loc[df['entry_date'] == '2019-02-29', 'entry_date'] = '2019-02-28'

In [10]:
df["pk_partition"]=pd.to_datetime(df["pk_partition"], format='%Y-%m-%d')
df["entry_date"]=pd.to_datetime(df["entry_date"], format='%Y-%m-%d')

Columns that only have a 0 or a 1 as a values, can be convert to int8 to save memory and computational costs. Because we have some nulls values we need do this step later, once we have dealt with all the missing data.

In [11]:
columns_to_convert = ['short_term_deposit', 'loans', 'mortgage', 'funds', 'securities', 'long_term_deposit', 'credit_card',
                      'payroll','pension_plan', 'payroll_account', 'emc_account', 'debit_card', 'em_account_p', 'active_customer']

In [12]:
for column in columns_to_convert:
    df[column] = df[column].astype('int8')

IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

#### **3.2 Integration**

##### **Data grouping**

When we have many different values for the same feature, the models may not be as efficient. It is more optimal if instead of having many values for a single feature, we have fewer. We can create new values or group those less frequent values into one.
First of all let's see what features we can apply this procedure:

* **country_id:** ES (Spain) has 5,960,672 of the total registrations, that is a 99.96% of the total. We can separate the data into "Spain" and "Others."

In [13]:
df['country_id'].value_counts()

country_id
ES    5960672
GB        441
FR        225
DE        199
US        195
CH        194
BR         87
BE         81
VE         79
IE         68
MX         58
AT         51
AR         51
PL         49
IT         45
MA         34
CL         30
CN         28
CA         22
LU         17
ET         17
QA         17
CI         17
SA         17
CM         17
SN         17
MR         17
NO         17
RU         17
CO         17
GA         17
GT         17
DO         17
SE         16
DJ         11
PT         11
JM         11
RO          9
HU          8
DZ          7
PE          4
Name: count, dtype: int64

We add all the values that are not Spain in a list, and then we regroup all these values by the value "OTHERS". We finally see how the values of the "country_id" feature look.

In [14]:
others_country = df['country_id'].value_counts()[df['country_id'].value_counts() < 500 ].index.tolist()
df.loc[df['country_id'].isin(others_country), 'country_id'] = 'OTHERS'

In [15]:
df['country_id'].value_counts()

country_id
ES        5960672
OTHERS       2252
Name: count, dtype: int64

* **entry_channel:** To optimize the performance of the model it is advisable to regroup the variables. We will use the same criteria that we have used in Section 2 --> EDA, Analyze by visualizing data,  where we have visualized the entry_channel variable regrouping those values that have a frequency less than 2% of the total values.

We see how we have too many values for a single feature.

In [18]:
df['entry_channel'].value_counts()

entry_channel
KHE    3113947
KFC     890620
KHQ     590280
KAT     416084
KHK     230197
KHM     176591
KHN     108434
KFA      79020
KHD      75899
RED      60601
KHL      41736
KHF      20657
KHO       8247
KAZ       5635
KHC       5241
KBG       1662
KEH        728
KHP        691
007        613
013        397
KAG        335
KAS        305
KBZ        198
KAA        193
KAR        157
KAY        149
KAF        123
KAB         95
KFD         91
KCC         89
KAW         65
KAQ         52
KDH         51
KAE         48
KCH         48
KCB         47
KAD         45
KAM         43
KAH         40
KAK         34
KAJ         33
KDT         23
KFF         22
KBO         21
KEY         20
KGC         17
004         17
KFS         17
KCL         17
KBE         17
KBH         17
KFK         17
KGN         17
KCI         17
KGX         17
KDR         16
KFL         13
KBW         12
KBU         11
KAI         11
KCK         11
KBY         11
KES         11
KEJ          8
KHS          5
KDA        

In [22]:
# We find values with a frequency of less than 0.02% of the total
value_counts_entry_channel = df['entry_channel'].value_counts(normalize = True)
others_entry_channel = value_counts_entry_channel[value_counts_entry_channel<0.02].index.tolist()

# Data grouping
df['entry_channel'] = df['entry_channel'].apply(lambda x: 'OTHERS' if x in others_entry_channel else x)

In [26]:
df['entry_channel'].value_counts()

entry_channel
KHE       3113947
KFC        890620
KHQ        590280
KAT        416084
OTHERS     412172
KHK        230197
KHM        176591
Name: count, dtype: int64

#### **Handling missing values**

We need to fill in the following features:

In [39]:
# number of nulls
for i in df.columns.values:
    if df[i].isnull().sum()>0:
        print(f"Total sum of nulls of {i} --->",df[i].isnull().sum())

Total sum of nulls of payroll ---> 61
Total sum of nulls of pension_plan ---> 61
Total sum of nulls of entry_channel ---> 133033
Total sum of nulls of segment ---> 133944
Total sum of nulls of region_code ---> 2264
Total sum of nulls of gender ---> 25
Total sum of nulls of salary ---> 1512103


* **salary:**

In [43]:
(df['salary'].isnull().sum()/df.shape[0])*100

25.358414764300196

* **entry_channel:** For Imputing the null values present in the categorical columns we used mode impuation. However "entry_channel" has 2.23% of missing values, which is large enough to change the distribution of the data, which we don't want. So we will impute the values according to the class weights to keep the data distribution uniform.

In [46]:
((df['entry_channel'].isnull().sum()/df.shape[0])*100)

2.2310027764901914

In [45]:
df['entry_channel'].value_counts(dropna = False)

entry_channel
KHE       3113947
KFC        890620
KHQ        590280
KAT        416084
OTHERS     412172
KHK        230197
KHM        176591
NaN        133033
Name: count, dtype: int64

* **segment:**

* **region_code:**

* **gender:**

* **payroll , pension_plan:****

#### **Creating new variables:**

#### **Encoding categorical variables:**

#### **Studying correlations:**

#### **Studying variance:**